## Part 1: Obtaining & Scrubbing Data
Go Back | Go Forward

### Table of Contents
#### 1. Obtaining Data   
    I. Yelp   
    II. NYC Open Data   
 
 
#### 2. Scrubbing Data    
    I.    
    II.   

### 1. Obtaining Data
#### I. Yelp
First I'm going to use the Yelp API to scrape the business information for restraurants in Manhattan. I'm going to ask Yelp to return 500 results per neighborhood. I got my list of neighborhoods from https://data.beta.nyc/dataset/pediacities-nyc-neighborhoods. 

In [1]:
import numpy as np
import pandas as pd

I wrote some general functions to use with the Yelp API that can be viewed in [project_functions2]. The code below is iterating through the list of neighborhoods. For each neighborhood, pf.get_businesses() is executed to return a JSON object with the Yelp Business Search data. The data is then transformed into a pandas data frame. Next, pf.clean_yelp_df() is performing a number of operations to unpack the data from the JSON format. The modified data frame is concatenated to the newly established main data frame, yelp_data, and saved in a CSV file under the name of the neighborhood. Finally, yelp_data is also saved to a CSV.

In [5]:
nyc_zips = pd.read_csv('nyc_zips.csv')

#### II. NYC Open Data


In [6]:
import project_functions2 as pf

In [7]:
df = pd.read_csv('yelp_data_master.csv')

I want to add some additional information to the features, and the nyc_zips dataframe I read in earlier can be applied to add features like, the name of the neighborhood, the borough, population by zip code and population density by zipcode. I'm going to make a dictionary for each of these features and use .map to apply the values using zip code.

In [8]:
nyc_neighborhood_zips = nyc_zips.set_index('zip').neighborhood.to_dict()
nyc_boro_by_zip = nyc_zips.set_index('zip').borough.to_dict()
pop_by_neighborhood = nyc_zips.set_index('zip').population.to_dict()
density_by_neighborhood = nyc_zips.set_index('zip').density.to_dict()

df['neighborhood_by_zip'] = df.zip_code.map(nyc_neighborhood_zips)
df['boro'] = df.zip_code.map(nyc_boro_by_zip)
df['pop_by_zip'] = df.zip_code.map(pop_by_neighborhood)
df['density_by_zip'] = df.zip_code.map(density_by_neighborhood)

I'm going to use the Restaurant Inspection Data CSV because it has a zip code column and an NTA column. I can use this to create another dictionary and also add NTA as feature.

In [9]:
nyc_open_df = pd.read_csv('data/NYC_OpenData.csv')
nyc_open_df = nyc_open_df.dropna()
nyc_open_df = nyc_open_df[nyc_open_df.BORO == 'Manhattan']
nyc_open_df['ZIPCODE'] = nyc_open_df.ZIPCODE.drop_duplicates()
nyc_open_df['ZIPCODE'] = nyc_open_df.ZIPCODE.astype(int).astype(str)
df['zip_code'] = df.zip_code.astype(str)
nyc_zips_by_nta = nyc_open_df.set_index('ZIPCODE').NTA.to_dict()
df['NTA'] = df.zip_code.astype(str).map(nyc_zips_by_nta)

In [10]:
len(df.columns)

484

Since I have so many columns, I want to take a look at what they actually are before deciding if I should keep them. The reason there are so many is becuase for each of the 3000+ restaurants on our list, each of them can have up to three categories. The way that I unpacked and encoded them, created a column for each one, even if there was only 1 instance, or ultimately zero instances if that observation was eliminated in some kind of way. So I'm going to examine the columns and their value counts and keep only the ones with 10+ observations.

In [11]:
list(df.columns)

['review_count',
 'distance',
 'airportlounges_cat_1',
 'australian_cat_1',
 'austrian_cat_1',
 'bagels_cat_1',
 'bakeries_cat_1',
 'bars_cat_1',
 'beerbar_cat_1',
 'belgian_cat_1',
 'boating_cat_1',
 'brasseries_cat_1',
 'brazilian_cat_1',
 'breakfast_brunch_cat_1',
 'british_cat_1',
 'burgers_cat_1',
 'cafes_cat_1',
 'cajun_cat_1',
 'caribbean_cat_1',
 'catering_cat_1',
 'chinese_cat_1',
 'cocktailbars_cat_1',
 'coffee_cat_1',
 'coffeeroasteries_cat_1',
 'comedyclubs_cat_1',
 'comfortfood_cat_1',
 'creperies_cat_1',
 'cuban_cat_1',
 'delis_cat_1',
 'desserts_cat_1',
 'diners_cat_1',
 'donuts_cat_1',
 'filipino_cat_1',
 'food_court_cat_1',
 'french_cat_1',
 'gastropubs_cat_1',
 'georgian_cat_1',
 'german_cat_1',
 'gourmet_cat_1',
 'greek_cat_1',
 'grocery_cat_1',
 'himalayan_cat_1',
 'hotels_cat_1',
 'hungarian_cat_1',
 'irish_cat_1',
 'irish_pubs_cat_1',
 'italian_cat_1',
 'japanese_cat_1',
 'juicebars_cat_1',
 'korean_cat_1',
 'latin_cat_1',
 'lounges_cat_1',
 'meats_cat_1',
 'medit

I'm going to seperate all of the 'cat_' columns:

In [12]:
old_cat_cols = [col for col in df.columns if str(col).endswith('cat_1') or str(col).endswith('cat_2') or str(col).endswith('cat_3')]

Now I'm going to reduce this list to only the category features that have 10 or more observations.

In [13]:
cat_df = df[old_cat_cols]
cat_df.loc['sum'] = cat_df.sum()
cat_sums = pd.DataFrame(cat_df.loc['sum'])
cat_sums = cat_sums[cat_sums['sum'] >= 10]
cat_sums = cat_sums.sort_values(by='sum')
cat_sums = cat_sums.T
cat_cols = list(cat_sums.columns)
new_cat_cols = [col for col in df.columns if col in cat_cols]

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Now, let's take a look at all the non-category columns.

In [14]:
other_cols = [col for col in df.columns if col not in old_cat_cols]

In [15]:
other_cols

['review_count',
 'distance',
 'lat',
 'long',
 'delivery',
 'pickup',
 'reservation',
 'price_1',
 'price_2',
 'price_3',
 'price_4',
 'address1',
 'city',
 'zip_code',
 'display_address',
 'above_avg',
 'neighborhood_by_zip',
 'boro',
 'pop_by_zip',
 'density_by_zip',
 'NTA']

I want to leave some of the geographical features like 'city' and 'zip_code' because they might come in useful later for visualizations or analysis. But features like 'distance' and 'boro' will be problematic. This is becuase the distance calculated is from arbitrary coordinates generated to represent the neighborhood we searched.

In [20]:
other_cols.remove('distance')
other_cols.remove('boro')

Now we can concatenate the two lists of reduced columns and restric the data frame to that

In [21]:
print('Length of new_cat_cols: ', len(new_cat_cols))
print('Length of other_cols: ', len(other_cols))
feature_cols = new_cat_cols + other_cols
print('Length of feature_cols: ', len(feature_cols))
df = df[feature_cols]
print('Length of df: ', len(df))

Length of new_cat_cols:  124
Length of other_cols:  19
Length of feature_cols:  143
Length of df:  3628


With our new modified data frame we move forward to check for missing values. Because there are many columns, and the nulls in the category data are going to be zeros, we need to check the columns that fall into 'other_cols'.

In [30]:
df_sums = pd.DataFrame(df[other_cols].isna().sum()).sort_values(by=0, ascending=False)
display(df_sums)

,0
NTA,1096
density_by_zip,328
pop_by_zip,328
neighborhood_by_zip,328
address1,2
above_avg,0
display_address,0
zip_code,0
city,0
review_count,0


It seems like there is a lot NTA data missing, probably due to a zip code problem since that was used to fill in the NTA columns.

In [38]:
df_zips = sorted(list(df[df.NTA.isna()].zip_code.unique()))

In [47]:
print(type(df_zips[0]))

<class 'str'>


In [48]:
sorted(df_zips)

['10166',
 '10175',
 '10270',
 '10301',
 '10302',
 '10304',
 '10310',
 '10314',
 '10452',
 '10454',
 '10455',
 '10458',
 '10459',
 '10461',
 '10462',
 '10463',
 '10464',
 '10466',
 '10468',
 '10471',
 '10473',
 '10474',
 '10475',
 '10801',
 '10803',
 '11101',
 '11102',
 '11103',
 '11104',
 '11105',
 '11106',
 '11109',
 '11201',
 '11204',
 '11205',
 '11206',
 '11207',
 '11208',
 '11209',
 '11211',
 '11213',
 '11214',
 '11215',
 '11216',
 '11217',
 '11218',
 '11219',
 '11220',
 '11221',
 '11222',
 '11225',
 '11226',
 '11228',
 '11229',
 '11230',
 '11231',
 '11232',
 '11233',
 '11234',
 '11237',
 '11238',
 '11249',
 '11354',
 '11355',
 '11356',
 '11357',
 '11358',
 '11360',
 '11361',
 '11365',
 '11366',
 '11368',
 '11370',
 '11371',
 '11372',
 '11373',
 '11374',
 '11375',
 '11377',
 '11378',
 '11379',
 '11385',
 '11414',
 '11415',
 '11418',
 '11419',
 '11420',
 '11421',
 '11434',
 '7002',
 '7010',
 '7011',
 '7012',
 '7013',
 '7020',
 '7022',
 '7024',
 '7026',
 '7029',
 '7030',
 '7031',
 '

Some of these may be Manhattan zips, so we will check them against the list.

In [49]:
print(type(mn_zips[0]))

<class 'str'>


In [40]:
mn_zips = list(nyc_open_df.ZIPCODE.unique())

In [45]:
sorted(mn_zips)

['10000',
 '10001',
 '10002',
 '10003',
 '10004',
 '10005',
 '10006',
 '10007',
 '10009',
 '10010',
 '10011',
 '10012',
 '10013',
 '10014',
 '10016',
 '10017',
 '10018',
 '10019',
 '10020',
 '10021',
 '10022',
 '10023',
 '10024',
 '10025',
 '10026',
 '10027',
 '10028',
 '10029',
 '10030',
 '10031',
 '10032',
 '10033',
 '10034',
 '10035',
 '10036',
 '10037',
 '10038',
 '10039',
 '10040',
 '10041',
 '10044',
 '10048',
 '10055',
 '10065',
 '10069',
 '10075',
 '10080',
 '10103',
 '10105',
 '10106',
 '10107',
 '10111',
 '10112',
 '10115',
 '10118',
 '10119',
 '10121',
 '10123',
 '10128',
 '10153',
 '10154',
 '10155',
 '10158',
 '10165',
 '10167',
 '10168',
 '10169',
 '10170',
 '10171',
 '10172',
 '10173',
 '10174',
 '10176',
 '10177',
 '10178',
 '10179',
 '10271',
 '10279',
 '10280',
 '10281',
 '10282',
 '12345']

In [41]:
common_zips = [z for z in mn_zips if z in df_zips]

In [42]:
common_zips

[]

After I didn't find any common zips, and investigated some of the complete zips, I discovered that some of them are for Queens, Brooklyn, the Bronx, and Staten Island. Since our analysis is limited to Manhattan, we want to exclude those. In addition there are a number of zips that I can't guess where they come from because they are incomplete. As a result, we may have to elimiate the blank NTA columns.

In [55]:
df = df[df.NTA.notna()]
df_sums = pd.DataFrame(df[other_cols].isna().sum()).sort_values(by=0, ascending=False)
display(df_sums)

,0
density_by_zip,22
pop_by_zip,22
neighborhood_by_zip,22
address1,2
review_count,0
above_avg,0
display_address,0
zip_code,0
city,0
price_4,0


Now our list of missing data is less problematic. Missing items in 'neighborhood_by_zip' is not a huge problem if we have the NTA. In addition, density by zip, and pop by zip can be filled with the averages.

In [60]:
avg_density_by_zip = df.density_by_zip.mean()
df['density_by_zip'] = df.density_by_zip.fillna(avg_density_by_zip)
avg_pop_by_zip = df.pop_by_zip.mean()
df['pop_by_zip'] = df.pop_by_zip.fillna(avg_pop_by_zip) 
df['neighborhood_by_zip'] = df.neighborhood_by_zip.fillna('missing')
df['address1'] = df.address1.fillna('missing')

In [61]:
df = df[df.NTA.notna()]
df_sums = pd.DataFrame(df[other_cols].isna().sum()).sort_values(by=0, ascending=False)
display(df_sums)

,0
review_count,0
address1,0
density_by_zip,0
pop_by_zip,0
neighborhood_by_zip,0
above_avg,0
display_address,0
zip_code,0
city,0
price_4,0


Finally, we can fill in the OHE features with missing values by using zero.

In [63]:
df = df.fillna(0)

Now that these are no more missing values, we can move forward for some exploratory analysis.

In [64]:
df.to_csv('scrubbed_data.csv', index=False)